<a href="https://colab.research.google.com/github/lexie-z/Introduction-to-ML-Fall2021/blob/main/mean_field_DOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
import torch.nn as nn
np.random.seed(234198)

import scipy.stats

class stock:
  #simulating M populations of N agent with NT+1 stopping times 
    def __init__(self, T, K, sigma, delta, So, r, NT, M, N):
        self.T = T
        self.K=K
        self.sigma=sigma *np.ones(N)
        self.delta=delta
        self.So=So*np.ones(N)
        self.r=r
        self.NT=NT
        self.M=M
        self.N=N
    
    # generate NT+1 by M by N state matrix
    def GBM(self):

        dt=self.T/self.NT
        So_vec=self.So*np.ones((1,S.M, S.N))
        
        Z=np.random.standard_normal((self.NT,self.M, self.N))
        s=self.So*np.exp(np.cumsum((self.r-self.delta-0.5*self.sigma**2)*dt+self.sigma*np.sqrt(dt)*Z, axis=0))
        
        s=np.append(So_vec, s, axis=0)
        return s
    
    # recover the histogram mu_n^m from the N agents 
    def mean(self,n,x,G):
        diag_G = torch.diag(G).float()
        # 0 for continue, 1 for stop, look for continuing players
        refined = torch.sub(torch.eye(self.M).float(),diag_G)
        continued = torch.matmul(refined.float(),x[int(n),:,:])
        filtered_continued = continued[continued.sum != 0]
        xmean = filtered_continued.mean(dim=0).float()
        return xmean
    
    def g(self,n,m,x,G):
        xmean = self.mean(n,x,G)
        max1=torch.max(x[int(n),m,:].float()-xmean.float())
        
        return np.exp(-self.r*(self.T/self.N)*n)*torch.max(max1,torch.tensor([0.0]))
       

#%%
class NeuralNet(torch.nn.Module):
    def __init__(self, d, q1, q2):
        super(NeuralNet, self).__init__()
        self.a1 = nn.Linear(2*d, q1) 
        self.relu = nn.ReLU()
        self.a2 = nn.Linear(q1, q2)
        self.a3 = nn.Linear(q2, 1)  
        self.sigmoid=nn.Sigmoid()
    
    def forward(self, x):
        out = self.a1(x)
        out = self.relu(out)
        out = self.a2(out)
        out = self.relu(out)
        out = self.a3(out)
        out = self.sigmoid(out)
        
        return out
    
def loss(y_pred,s, x, n, tau,G):
    # n = time step
    # s.M = number of agents
    r_n=torch.zeros((s.M))
    for m in range(0,s.M):
        
        r_n[m]=-s.g(n,m,x,G)*y_pred[m] - s.g(tau[m],m,x,G)*(1-y_pred[m])
    
    return(r_n.mean())
    
#%%

S=stock(3,100,0.2,0.1,90,0.05,9,500,10)   
X=torch.from_numpy(S.GBM()).float()
print(X.size())

#%%
def NN(n,x,s, tau_n_plus_1,G):       
    epochs=50
    model=NeuralNet(s.N,s.N+40,s.N+40)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    
    
    Xmean = S.mean(n,X,G)
    Xmean_prime = Xmean.expand(S.M,S.N)
    MF_input = torch.cat((X[n],Xmean_prime),1)
    

    for epoch in range(epochs):
        F = model.forward(MF_input)
        optimizer.zero_grad()
        criterion = loss(F,S,X,n,tau_n_plus_1,G)
        criterion.backward()
        optimizer.step()
    
    return F,model

mods=[None]*S.NT
tau_mat=np.zeros((S.NT+1,S.M))
tau_mat[S.NT,:]=S.NT

f_mat=np.zeros((S.NT+1,S.M))
f_mat[S.NT,:]=1

#%%
for n in range(S.NT-1,-1,-1):
    G = torch.from_numpy(f_mat[n+1]).float()
    probs, mod_temp=NN(n, X, S,torch.from_numpy(tau_mat[n+1]).float(),G)
    mods[n]=mod_temp
    
    np_probs=probs.detach().numpy().reshape(S.M)
    print(n, ":", np.min(np_probs)," , ", np.max(np_probs))

    f_mat[n,:]=(np_probs > 0.5)*1.0

    tau_mat[n,:]=np.argmax(f_mat, axis=0)

#%% 
Y=torch.from_numpy(S.GBM()).float()

tau_mat_test=np.zeros((S.NT+1,S.M))
tau_mat_test[S.NT,:]=S.NT

f_mat_test=np.zeros((S.NT+1,S.M))
f_mat_test[S.NT,:]=1

V_mat_test=np.zeros((S.NT+1,S.M))
V_est_test=np.zeros(S.NT+1)

G_end = torch.from_numpy(f_mat_test[S.NT]).float()
for m in range(0,S.M):
  V_mat_test[S.NT,m]=S.g(S.NT,m,Y,G_end)
    
V_est_test[S.NT]=np.mean(V_mat_test[S.NT,:])



for n in range(S.NT-1,-1,-1):
  mod_curr=mods[n]

  G_Y = torch.from_numpy(f_mat_test[n+1]).float()
  Ymean = S.mean(n,Y,G_Y)
  Ymean_prime = Ymean.expand(S.M,S.N)
  MF_input = torch.cat((Y[n],Ymean_prime),1)

  probs=mod_curr(MF_input)
  np_probs=probs.detach().numpy().reshape(S.M)

  f_mat_test[n,:]=(np_probs > 0.5)*1.0

  tau_mat_test[n,:]=np.argmax(f_mat_test, axis=0)

  for m in range(0,S.M):
      V_mat_test[n,m]=np.exp((n-tau_mat_test[n,m])*(-S.r*S.T/S.N))*S.g(tau_mat_test[n,m],m,X,G_Y) 
        

#%%
V_est_test=np.mean(V_mat_test, axis=1)
V_std_test=np.std(V_mat_test, axis=1)
V_se_test=V_std_test/(np.sqrt(S.M))

z=scipy.stats.norm.ppf(0.975)
lower=V_est_test[0] - z*V_se_test[0]
upper=V_est_test[0] + z*V_se_test[0]

print(V_est_test[0])
print(V_se_test[0])
print(lower)
print(upper)

torch.Size([10, 500, 10])


KeyboardInterrupt: ignored